In [ ]:
# main.py
import sys
from PySide6.QtWidgets import QApplication
from app.core.player import SubtitlePlayer
from app.core.view import SubtitleWindow

if __name__ == "__main__":
    app = QApplication(sys.argv)

    player = SubtitlePlayer("script.json")   # 非 GUI 核心
    window = SubtitleWindow(player)          # GUI 只订阅信号
    window.show()

    sys.exit(app.exec())


In [ ]:
# main_audio_demo.py
from PySide6.QtWidgets import QApplication
from app.core.audio.audio_hub import AudioHub
import sys, numpy as np

app  = QApplication(sys.argv)
hub  = AudioHub(channels=1)          # 未来换成 channels=4 即可
hub.blockReady.connect(
    lambda ch, blk: print(f"ch{ch} rms={np.sqrt((blk**2).mean()):.3f}")
)
hub.start()
sys.exit(app.exec())


In [ ]:
# test_live_stt.py
import sys, signal
import numpy as np
from PySide6.QtWidgets import QApplication
from app.core.audio.audio_hub import AudioHub            # 你的“耳朵”
from app.core.stt.vosk_engine import VoskEngine          # 参照接口实现
from app.core.stt.base   import TranscriptPiece


# ---------- 1. Qt 应用 ----------
app = QApplication(sys.argv)                    # 仅为用 Qt Signals

# ---------- 2. 配置模块 ----------
hub  = AudioHub(channels=1,
                samplerate=16000,  # 采样率 16kHz
                frames_per_block=3200) # 采样率 16kHz，单通道，6400 帧每块（400ms）  
engine  = VoskEngine(model_dir="F:\Miomu\Miomu\\app\models\stt\\vosk\\vosk-model-cn-0.22", lang= 'zh',channel_id=0)

# ---------- 3. 信号接线 ----------
def on_audio_block(ch: int, block: np.ndarray):
    if ch == 0:                                 # 单通道 MVP
        engine.feed(block)

def on_transcript(ch: int, piece: TranscriptPiece):
    print(f"[ch{ch}] {piece.text} (conf={piece.confidence:.2f})")

hub.blockReady.connect(on_audio_block)
engine.segmentReady.connect(on_transcript)

# ---------- 4. 启动 ----------
engine.start()
hub.start()

print("📢 Speak into the mic. Press Ctrl+C to exit.")
signal.signal(signal.SIGINT, lambda *_: app.quit())  # Ctrl+C → 退出

sys.exit(app.exec())     # Qt 事件循环


[AudioHub] Started, 1 ch @ 16000 Hz
📢 Speak into the mic. Press Ctrl+C to exit.
[VoskEngine] recognizer ready
[ch0] 你 看 (conf=0.50)
[ch0] 你 看 把 (conf=0.50)
[ch0] 你 看 把 这个 (conf=0.50)
[ch0] 你 看 把 这个 就 (conf=0.50)
[ch0] 你 看 把 这个 就 挺 好 (conf=0.50)
[ch0] 你 看 把 这个 就 挺 好的 (conf=0.50)
[ch0] 你 看 把 这个 就 挺 好的 真的 (conf=0.50)
[ch0] 你 看 把 这个 就 挺 好的 真的 那个 (conf=0.50)
[ch0] 你 看 把 这个 就 挺 好的 真的 那个 比例 (conf=0.50)
[ch0] 你 看 把 这个 就 挺 好的 真的 那个 比 那个 (conf=0.50)
[ch0] 你 看 把 这个 就 挺 好的 真的 那个 比例 (conf=0.50)
[ch0] 你 看 把 这个 就 挺 好的 真的 那个 比例 个 (conf=0.50)
[ch0] 你 看 把 这个 就 挺 好的 真的 那个 比 那个 该死 (conf=0.50)
[ch0] 你 看 把 这个 就 挺 好的 真的 那个 比 那个 该死 的 (conf=0.50)
[ch0] 你 看 把 这个 就 挺 好的 真的 那个 比 那个 该死 的 也好 (conf=0.50)
[ch0] 你 看 把 这个 就 挺 好的 真的 那个 比 那个 该死 的 要好 很多 (conf=0.50)
[ch0] 你 看 把 这个 就 挺 好的 真的 那个 比 那个 该死 的 要好 很多 的 (conf=0.50)
[ch0] 你 看 把 这个 就 挺 好的 真的 那个 比 那个 该死 的 要好 很多 的 且 (conf=0.50)
[ch0] 你 看 把 这个 就 挺 好的 真的 那个 比 那个 该死 的 要好 很多 而且 速度 (conf=0.50)
[ch0] 你 看 把 这个 就 挺 好的 真的 那个 比 那个 该死 的 要好 很多 而且 速度 也 (conf=0.50)
[ch0] 你 看 把 这个 就 

In [ ]:
import os, sys
os.add_dll_directory(
    r"C:\Program Files\NVIDIA\CUDNN\v9.1\bin\12.4")
import sys, signal, numpy as np
from PySide6.QtCore    import QCoreApplication          # 轻量级，无 GUI
from app.core.audio.audio_hub     import AudioHub
from app.core.stt.whisper_engine  import WhisperEngine


app = QCoreApplication(sys.argv)


hub     = AudioHub(channels=1, samplerate=16_000, frames_per_block=3200,silence_thresh=0.03)
engine  = WhisperEngine(model_size="F:\Miomu\Miomu\\app\models\stt\\faster-whisper\\faster-whisper-small", device="cpu", language="zh")


hub.blockReady.connect(lambda ch, blk: engine.feed(blk))
engine.segmentReady.connect(lambda ch, piece: print(f"[ch{ch}] {piece.text}"))

engine.start()
hub.start()

print("🎤 Speak into the mic.  Ctrl-C to exit.")
signal.signal(signal.SIGINT, lambda *_: app.quit())

sys.exit(app.exec())           # ← 关键：Qt 事件循环
